In [1]:
import pandas as pd
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.options.display.float_format = '{:,.2f}'.format #отображение данных на уровне 2-х знаков
pd.set_option('display.max_columns', None) #отображение всех столбцов датафрейма

In [3]:
data = pd.read_excel('/Users/REKL1/Downloads/TLV/RKC_02_2023.xls') #загружаем 2021 год

In [5]:
print('Введите данные для контроля: общая сумма начисленных средств')
X_0 = float(input())

Введите данные для контроля: общая сумма начисленных средств
658375.38


In [6]:
print('Введите месяц загрузки')
MONTH = int(input())

Введите месяц загрузки
2


In [7]:
print('Введите год загрузки')
YEAR = int(input())

Введите год загрузки
2023


In [8]:
#data.head()

In [9]:
data = data.rename(columns = {'Справка о начислениях и оплате по услугам за период по лицевым счетам здания': 'Unnamed_0'})

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18903 entries, 0 to 18902
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Unnamed_0    18899 non-null  object        
 1   Unnamed: 1   18837 non-null  object        
 2   Unnamed: 2   7785 non-null   object        
 3   Unnamed: 3   7785 non-null   object        
 4   Unnamed: 4   1 non-null      object        
 5   Unnamed: 5   18837 non-null  object        
 6   Unnamed: 6   13342 non-null  object        
 7   Unnamed: 7   0 non-null      float64       
 8   Unnamed: 8   13343 non-null  object        
 9   Unnamed: 9   13342 non-null  object        
 10  Unnamed: 10  18838 non-null  object        
 11  Unnamed: 11  1 non-null      datetime64[ns]
 12  Unnamed: 12  7785 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(11)
memory usage: 1.9+ MB


Первый контрольный замер данных:

In [11]:
x_k_1 = data[data['Unnamed_0'] == "Система видеонаблюдения"]['Unnamed: 9'].sum()
x_d_1 = data[data['Unnamed_0'] == "Обслуживание домофона"]['Unnamed: 9'].sum()
y_k_1 = data[data['Unnamed_0'] == "Система видеонаблюдения"]['Unnamed_0'].count()
y_d_1 = data[data['Unnamed_0'] == "Обслуживание домофона"]['Unnamed_0'].count()

print(f'В представленном файле {y_k_1 + y_d_1} записей на сумму {x_k_1 + x_d_1: ,.2f}')

В представленном файле 7784 записей на сумму  658,375.38


In [12]:
data_1 = data.copy()

In [13]:
def dom(Unnamed):
    if str(Unnamed).find(", д.") > 1:
        return Unnamed

In [14]:
data_1['dom'] = data_1['Unnamed_0'].apply(dom)

In [15]:
for i in range(1, len(data)):
    if str(data_1['Unnamed_0'][i]).find(", д.") == -1:
        data_1['dom'][i] = data_1['dom'][i-1]
            
    else:
        data_1['dom'][i] = data_1['Unnamed_0'][i]

In [16]:
def kv(Unnamed):
    if str(Unnamed).find("в.") >= 1:
        return Unnamed

In [17]:
data_1['kv'] = data_1['Unnamed_0'].apply(kv)

In [ ]:
for i in range(1, len(data)):
    if str(data_1['Unnamed_0'][i]).find("кв.") == -1:
        data_1['kv'][i] = data_1['kv'][i-1]
            
    else:
        data_1['kv'][i] = data_1['Unnamed_0'][i]

In [ ]:
def ls(Unnamed):
    if str(Unnamed).find("№") >= 0:
        return Unnamed

In [ ]:
data_1['ls'] = data_1['Unnamed: 10'].apply(ls)

In [ ]:
for i in range(1, len(data)):
    if str(data_1['Unnamed: 10'][i]).find("лс") == -1:
        data_1['ls'][i] = data_1['ls'][i-1]
            
    else:
        data_1['ls'][i] = data_1['Unnamed: 10'][i]

In [ ]:
#data_1

In [ ]:
data_1 = data_1.drop(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 7', 'Unnamed: 8',
                      'Unnamed: 11', 'Unnamed: 12'],
                     axis=1)

In [ ]:
data_1 = data_1.rename(columns = {'Unnamed_0': 'services', 'Unnamed: 5': 'tarif', 'Unnamed: 6': 'tarif_now',
                              'Unnamed: 9': 'sum_total', 'Unnamed: 10': 'balance'})

In [ ]:
products = data_1.query('services == "Система видеонаблюдения" or services == "Обслуживание домофона"').reset_index(drop=True)

Второй контрольный замер:

In [ ]:
x_k_2 = products[products['services'] == "Система видеонаблюдения"]['sum_total'].sum()
x_d_2 = products[products['services'] == "Обслуживание домофона"]['sum_total'].sum()
y_k_2 = products[products['services'] == "Система видеонаблюдения"]['services'].count()
y_d_2 = products[products['services'] == "Обслуживание домофона"]['services'].count()

print(f'В представленном файле {y_k_2 + y_d_2} записей на сумму {x_k_2 + x_d_2: ,.2f}')

In [ ]:
if round((X_0/(x_k_2 + x_d_2)),2) == 1 and (y_k_2 + y_d_1) == (y_k_1 + y_d_2) and (x_k_2 + x_d_1) == (x_k_1 + x_d_2):
    print('Проверка завершена удачно')
else:
    print('Данные не достоверные!! Проверьте данные в файле example.xlsx')
    writer = pd.ExcelWriter('example.xlsx', engine='xlsxwriter') # Указать writer библиотеки
    products.to_excel(writer, 'Sheet1') # Записать ваш DataFrame в файл  
    writer.save() # Сохраним результат 

In [ ]:
#writer = pd.ExcelWriter('example.xlsx', engine='xlsxwriter') # Указать writer библиотеки
#control_1.to_excel(writer, 'Sheet1') # Записать ваш DataFrame в файл  
#writer.save() # Сохраним результат 

In [ ]:
products['services'] = products['services'].replace({'Обслуживание домофона': 'домофон', 'Система видеонаблюдения': 'камеры'})

In [ ]:
products['month'] = MONTH
products['year'] = YEAR

In [ ]:
products['dom_kv_PKC'] = 0
for i in range(len(products)):
        
    y = " ".join(products['kv'][i].split('. '))
    x = " ".join(products['dom'][i].split(', д.'))
            
    products['dom_kv_PKC'][i] = x + " " + y

In [ ]:
#products

In [ ]:
products.info()

In [ ]:
products['sum_total'] = products['sum_total'].astype(float)

In [ ]:
#products

Загружаем справочник лицевых счетов

In [ ]:
box_ls = pd.read_excel('/Users/REKL1/Downloads/TLV/ls_id_standart.xlsx', sheet_name='Sheet1') #загружаем лицевые счета

In [ ]:
#box_ls

In [ ]:
df = products.merge(box_ls, on=["ls", "services"], how = 'left').reset_index(drop=True)

In [ ]:
#df

In [ ]:
warning = 0
free_dom = []
for i in range(len(df)):
    if df['dom_kv_PKC'][i] != df['dom_kv'][i] and df['dom_kv_PKC'][i] != df['dom_kv'][i]:
        warning += 1
        free_dom.append(df['dom'][i])

In [ ]:
warning

In [ ]:
set(free_dom)

In [ ]:
control_1 = df.query('dom in @free_dom')

In [ ]:
writer = pd.ExcelWriter('control_1.xlsx', engine='xlsxwriter') # Указать writer библиотеки
control_1.to_excel(writer, 'Sheet1') # Записать ваш DataFrame в файл  
writer.save() # Сохраним результат 

In [ ]:
df[df['id'].isna() == True].query('dom == "СВЯТИТЕЛЯ ИННОКЕНТИЯ, д.19"').head(5)

In [ ]:
box_dom = pd.read_excel('/Users/REKL1/Downloads/TLV/ls_id_standart.xlsx', sheet_name='Лист2') #загружаем 2021 годa

In [ ]:
#box_dom.head(5)

In [ ]:
df_1 = df[df['id'].isna() == True]

In [ ]:
df_2 = df[df['id'].isna() == False]

In [ ]:
df_1 = df_1.merge(box_dom, on=["dom_kv_PKC", "services"], how = 'left').reset_index(drop=True)
df_1['id'] = df_1['cid']

In [ ]:
df_1[df_1['id'].isna() == True] #проверяем на пропуски

In [ ]:
df_1 = df_1[['dom', 'services', 'tarif', 'sum_total', 'tarif_now', 'balance', 'ls', 'month', 'year', 'dom_kv_PKC', 'id']]

In [ ]:
df_2 = df_2[['dom', 'services', 'tarif', 'sum_total', 'tarif_now', 'balance', 'ls', 'month', 'year', 'dom_kv_PKC', 'id']]

In [ ]:
data = df_1.append(df_2, sort = False).reset_index(drop=True) #объединяем файлы

In [ ]:
data.info()

Третья проверка:

In [ ]:
x_k_3 = data[data['services'] == "камеры"]['sum_total'].sum()
x_d_3 = data[data['services'] == "домофон"]['sum_total'].sum()
y_k_3 = data[data['services'] == "камеры"]['sum_total'].count()
y_d_3 = data[data['services'] == "домофон"]['sum_total'].count()

print(f'В представленном файле {y_k_3 + y_d_3} записей на сумму {x_k_3 + x_d_3: ,.2f}')

In [ ]:
data_export = data[data['sum_total'] > 0]
data_export = data_export[['ls', 'services','sum_total', 'id']].reset_index()

In [ ]:
data_export_1 = data[data['tarif_now'] != 0]
data_export_1 = data_export_1[['ls', 'services','tarif_now', 'id']].reset_index()

In [ ]:
#data_export_1

Выгружаем данные:

In [ ]:
writer = pd.ExcelWriter('data_PKC_01_23.xlsx', engine='xlsxwriter') # Указать writer библиотеки
data.to_excel(writer, 'Sheet1') # Записать ваш DataFrame в файл  
writer.save() # Сохраним результат 

In [ ]:
#writer = pd.ExcelWriter('ls_id.xlsx', engine='xlsxwriter') # Указать writer библиотеки
#data[['ls', 'services', 'id', 'dom']].to_excel(writer, 'Sheet1') # Записать ваш DataFrame в файл  
#writer.save() # Сохраним результат 

In [ ]:
writer = pd.ExcelWriter('data_export.xlsx', engine='xlsxwriter') # Указать writer библиотеки
data_export.to_excel(writer, 'Sheet1') # Записать ваш DataFrame в файл  
data_export_1.to_excel(writer, 'Sheet2') # Записать ваш DataFrame в файл  
writer.save() # Сохраним результат 